## Differential evolution (DE) as an optimizer for training a NN

In [2]:
# importing required libraries
import numpy as np
import pandas as pd

import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# importing library for plotting
import matplotlib.pyplot as plt

# importing library for support vector machine classifier
from sklearn.svm import SVC
# importing library for K-neares-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier
# importing library for Linear Discriminant Analysis Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# importing library for Quadratic Discriminant Analysis Model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

from keras.layers import Dense # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.models import model_from_json # saving and loading trained model

from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model

# representation of model layers
from keras.utils.vis_utils import plot_model
from numpy.random import choice as np_choice

In [1]:
path="datasets/bin_data.csv"
bin_data=pd.read_csv(path)
bin_data

NameError: name 'pd' is not defined

In [52]:
X = bin_data.iloc[:,0:93].values # dataset excluding target attribute (encoded, one-hot-encoded,original)
Y = bin_data[['intrusion']].values # target attribute

In [53]:
# splitting the dataset 75% for training and 25% testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)


In [54]:
def setWeights(X):
 Len=0
 for i in range(0,len(mlp.layers)):
   a=mlp.layers[i].get_weights()[0].shape
   weights=X[Len:(Len+mlp.layers[i].get_weights()[0].flatten().shape[0])]
   weights=np.reshape(weights,a) 
   Len=Len+mlp.layers[i].get_weights()[0].flatten().shape[0]
   a=mlp.layers[i].get_weights()[1].shape
   bias=X[Len:(Len+mlp.layers[i].get_weights()[1].flatten().shape[0])]
   bias=np.reshape(bias,a) 
   Len=Len+mlp.layers[i].get_weights()[1].flatten().shape[0]
   mlp.layers[i].set_weights([weights,bias])


In [55]:
def Fobj():
    mlp = Sequential() # creating model
    # adding input layer and first layer with 50 neurons
    mlp.add(Dense(units=50, input_dim=X_train.shape[1], activation='relu'))
    # output layer with sigmoid activation
    mlp.add(Dense(units=1,activation='sigmoid'))
    return mlp

In [56]:
mlp=Fobj()
# defining loss function, optimizer, metrics and then compiling model
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
dim=0;
for i in range(len(mlp.get_weights())):
   dim=dim+mlp.get_weights()[i].flatten().shape[0]
N=20# Population size
X=np.zeros((N,dim))
Fit=np.ones(N)*float('inf')
bF=float('inf')
bPos=[];

In [57]:
L_bounds=-10;
U_bounds=10;
def initialization():
  for i in range(N):
    for j in range(dim):
        X[i][j]=(L_bounds+np.random.rand(1)*(U_bounds-L_bounds))
    setWeights(X[i,:])
    print("------------Solution ",i,"-------------")
    score=mlp.evaluate(X_train, y_train, verbose = 1) 
    Fit[i]= score[0]
    print(score[1])

In [58]:
import copy

class AntColony(object):

    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        """
        Args:
            distances (2D numpy.array): Square matrix of distances. Diagonal is assumed to be np.inf.
            n_ants (int): Number of ants running per iteration
            n_best (int): Number of best ants who deposit pheromone
            n_iteration (int): Number of iterations
            decay (float): Rate it which pheromone decays. The pheromone value is multiplied by decay, so 0.95 will lead to decay, 0.5 to much faster decay.
            alpha (int or float): exponenet on pheromone, higher alpha gives pheromone more weight. Default=1
            beta (int or float): exponent on distance, higher beta give distance more weight. Default=1

        Example:
            ant_colony = AntColony(german_distances, 100, 20, 2000, 0.95, alpha=1, beta=2)          
        """
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best 
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            print (shortest_path)
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path            
            self.pheromone = self.pheromone * self.decay            
        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start)) # going back to where we started    
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move


In [ ]:
distances = np.array([[np.inf, 2, 2, 5, 7],
                      [2, np.inf, 4, 8, 2],
                      [2, 4, np.inf, 1, 3],
                      [5, 8, 1, np.inf, 2],
                      [7, 2, 3, 2, np.inf]])

ant_colony = AntColony(distances, 1, 1, 100, 0.95, alpha=1, beta=1)
shortest_path = ant_colony.run()
print ("shorted_path: {}".format(shortest_path))



In [59]:
def check_boundaries(X):
    for j in range(dim):
        if X[j]>U_bounds:
            X[j]=(L_bounds+np.random.random(1)*(U_bounds-L_bounds))
        elif  X[j]<L_bounds:
            X[j]=(L_bounds+np.random.random(1)*(U_bounds-L_bounds))

In [60]:
runs=1
avgLoss=0
max1=float('inf')
bPos=[]
t=1
Max_iter=50;
for i in range(runs):
    de=DE()
    avgLoss=avgLoss+de[0]
    bPos=de[1]

------------Solution  0 -------------
2953/2953 [==============================] - 6s 2ms/step - loss: 477.1825 - accuracy: 0.5245
0.5244763493537903
------------Solution  1 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 45.9745 - accuracy: 0.7886
0.7885879278182983
------------Solution  2 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 172.3611 - accuracy: 0.5208
0.5207506418228149
------------Solution  3 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 63.8890 - accuracy: 0.7563
0.7563056349754333
------------Solution  4 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 48.4355 - accuracy: 0.8010
0.8010140061378479
------------Solution  5 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 280.4646 - accuracy: 0.4660
0.46604007482528687
------------Solution  6 -------------
2953/2953 [==============================] - 5s 2ms/step - loss:

Accuracy: 0.20441579818725586  Loss:  647.1053466796875
------------Solution  15 -------------
Accuracy: 0.7926735281944275  Loss:  114.79662322998047
------------Solution  16 -------------
Accuracy: 0.4013272821903229  Loss:  243.1868438720703
------------Solution  17 -------------
Accuracy: 0.2544163167476654  Loss:  267.87384033203125
------------Solution  18 -------------
Accuracy: 0.7579885721206665  Loss:  147.4517364501953
------------Solution  19 -------------
Accuracy: 0.87777179479599  Loss:  64.38691711425781
The best so-far Fitness value obtained by DE at iteration  3 == 20.883459091186523
------------Solution  0 -------------
Accuracy: 0.8202775120735168  Loss:  67.90998077392578
------------Solution  1 -------------
Accuracy: 0.5682744383811951  Loss:  232.79808044433594
------------Solution  2 -------------
Accuracy: 0.310524046421051  Loss:  436.28192138671875
------------Solution  3 -------------
Accuracy: 0.876850962638855  Loss:  40.81298828125
------------Solution  

Accuracy: 0.6646556258201599  Loss:  162.89903259277344
------------Solution  19 -------------
Accuracy: 0.5037521719932556  Loss:  316.6260986328125
The best so-far Fitness value obtained by DE at iteration  7 == 20.883459091186523
------------Solution  0 -------------
Accuracy: 0.8178325295448303  Loss:  105.55199432373047
------------Solution  1 -------------
Accuracy: 0.41746842861175537  Loss:  103.89115142822266
------------Solution  2 -------------
Accuracy: 0.7266905903816223  Loss:  119.40188598632812
------------Solution  3 -------------
Accuracy: 0.21871526539325714  Loss:  258.10577392578125
------------Solution  4 -------------
Accuracy: 0.4649287164211273  Loss:  497.3186340332031
------------Solution  5 -------------
Accuracy: 0.8533642292022705  Loss:  76.63444519042969
------------Solution  6 -------------
Accuracy: 0.22888684272766113  Loss:  1270.6719970703125
------------Solution  7 -------------
Accuracy: 0.4544925391674042  Loss:  514.6531372070312
------------Sol

Accuracy: 0.4767302870750427  Loss:  177.82154846191406
------------Solution  2 -------------
Accuracy: 0.7858465909957886  Loss:  162.08251953125
------------Solution  3 -------------
Accuracy: 0.4522698223590851  Loss:  518.069091796875
------------Solution  4 -------------
Accuracy: 0.7050667405128479  Loss:  167.24789428710938
------------Solution  5 -------------
Accuracy: 0.6026312708854675  Loss:  633.09814453125
------------Solution  6 -------------
Accuracy: 0.2877676486968994  Loss:  477.2532653808594
------------Solution  7 -------------
Accuracy: 0.14418019354343414  Loss:  1353.9327392578125
------------Solution  8 -------------
Accuracy: 0.8473946452140808  Loss:  46.882747650146484
------------Solution  9 -------------
Accuracy: 0.7782999277114868  Loss:  186.23965454101562
------------Solution  10 -------------
Accuracy: 0.49439558386802673  Loss:  435.343994140625
------------Solution  11 -------------
Accuracy: 0.5312926769256592  Loss:  510.96875
------------Solution

Accuracy: 0.8698546886444092  Loss:  108.03850555419922
------------Solution  6 -------------
Accuracy: 0.7914986610412598  Loss:  122.56560516357422
------------Solution  7 -------------
Accuracy: 0.14765186607837677  Loss:  656.1381225585938
------------Solution  8 -------------
Accuracy: 0.6638829708099365  Loss:  322.4237060546875
------------Solution  9 -------------
Accuracy: 0.5634267926216125  Loss:  381.8765563964844
------------Solution  10 -------------
Accuracy: 0.8234105110168457  Loss:  88.7413101196289
------------Solution  11 -------------
Accuracy: 0.7556493878364563  Loss:  138.1665802001953
------------Solution  12 -------------
Accuracy: 0.6114480495452881  Loss:  237.83505249023438
------------Solution  13 -------------
Accuracy: 0.47454988956451416  Loss:  151.9868927001953
------------Solution  14 -------------
Accuracy: 0.48104870319366455  Loss:  335.18017578125
------------Solution  15 -------------
Accuracy: 0.5038897395133972  Loss:  266.1126708984375
------

...

Accuracy: 0.8314334154129028  Loss:  67.64615631103516
------------Solution  1 -------------
Accuracy: 0.8901448845863342  Loss:  107.88418579101562
------------Solution  2 -------------
Accuracy: 0.5857703685760498  Loss:  73.37832641601562
------------Solution  3 -------------
Accuracy: 0.45162418484687805  Loss:  760.4951782226562
------------Solution  4 -------------
Accuracy: 0.5243175625801086  Loss:  271.4444580078125
------------Solution  5 -------------
Accuracy: 0.14408493041992188  Loss:  542.892333984375
------------Solution  6 -------------
Accuracy: 0.4690248668193817  Loss:  962.7926025390625
------------Solution  7 -------------
Accuracy: 0.5515193939208984  Loss:  83.62349700927734
------------Solution  8 -------------
Accuracy: 0.5539537668228149  Loss:  228.19430541992188
------------Solution  9 -------------
Accuracy: 0.5315255522727966  Loss:  412.62786865234375
------------Solution  10 -------------
Accuracy: 0.7482826709747314  Loss:  138.2008514404297
----------

Accuracy: 0.8028027415275574  Loss:  263.5298767089844
------------Solution  5 -------------
Accuracy: 0.5795996785163879  Loss:  624.6641845703125
------------Solution  6 -------------
Accuracy: 0.5660305619239807  Loss:  415.5393981933594
------------Solution  7 -------------
Accuracy: 0.8035436272621155  Loss:  79.39788055419922
------------Solution  8 -------------
Accuracy: 0.5662739872932434  Loss:  219.25137329101562
------------Solution  9 -------------
Accuracy: 0.8746282458305359  Loss:  53.38882064819336
------------Solution  10 -------------
Accuracy: 0.8082537055015564  Loss:  189.77088928222656
------------Solution  11 -------------
Accuracy: 0.7944516539573669  Loss:  180.18429565429688
------------Solution  12 -------------
Accuracy: 0.5323828458786011  Loss:  430.6666259765625
------------Solution  13 -------------
Accuracy: 0.5162945985794067  Loss:  200.04176330566406
------------Solution  14 -------------
Accuracy: 0.7991405725479126  Loss:  153.8297119140625
------

Accuracy: 0.5369341373443604  Loss:  159.0138397216797
------------Solution  9 -------------
Accuracy: 0.7372431755065918  Loss:  114.94031524658203
------------Solution  10 -------------
Accuracy: 0.7145926356315613  Loss:  124.82269287109375
------------Solution  11 -------------
Accuracy: 0.4062172472476959  Loss:  557.2654418945312
------------Solution  12 -------------
Accuracy: 0.8209125995635986  Loss:  73.63349151611328
------------Solution  13 -------------
Accuracy: 0.5043978095054626  Loss:  237.995849609375
------------Solution  14 -------------
Accuracy: 0.7342795729637146  Loss:  269.5004577636719
------------Solution  15 -------------
Accuracy: 0.8791477680206299  Loss:  130.74343872070312
------------Solution  16 -------------
Accuracy: 0.8361223340034485  Loss:  36.6993408203125
------------Solution  17 -------------
Accuracy: 0.6220747232437134  Loss:  330.0613098144531
------------Solution  18 -------------
Accuracy: 0.6974671483039856  Loss:  162.30455017089844
----

Accuracy: 0.5273764729499817  Loss:  283.2376708984375
------------Solution  13 -------------
Accuracy: 0.15103885531425476  Loss:  1568.259765625
------------Solution  14 -------------
Accuracy: 0.37556493282318115  Loss:  564.9713745117188
------------Solution  15 -------------
Accuracy: 0.9008033275604248  Loss:  58.43303680419922
------------Solution  16 -------------
Accuracy: 0.8882397413253784  Loss:  51.53981399536133
------------Solution  17 -------------
Accuracy: 0.6350829005241394  Loss:  330.4560241699219
------------Solution  18 -------------
Accuracy: 0.5533716678619385  Loss:  621.07666015625
------------Solution  19 -------------
Accuracy: 0.48801320791244507  Loss:  354.0299072265625
The best so-far Fitness value obtained by DE at iteration  45 == 20.883459091186523
------------Solution  0 -------------
Accuracy: 0.5590977668762207  Loss:  559.6851196289062
------------Solution  1 -------------
Accuracy: 0.6145915985107422  Loss:  448.448974609375
------------Solution

Accuracy: 0.45611193776130676  Loss:  432.5027160644531
------------Solution  18 -------------
Accuracy: 0.28856149315834045  Loss:  497.32232666015625
------------Solution  19 -------------
Accuracy: 0.6581674218177795  Loss:  192.94073486328125
The best so-far Fitness value obtained by DE at iteration  49 == 20.883459091186523
------------Solution  0 -------------
Accuracy: 0.77997225522995  Loss:  160.57168579101562
------------Solution  1 -------------
Accuracy: 0.528032660484314  Loss:  136.42738342285156
------------Solution  2 -------------
Accuracy: 0.785148024559021  Loss:  126.224609375
------------Solution  3 -------------
Accuracy: 0.46007049083709717  Loss:  129.95022583007812
------------Solution  4 -------------
Accuracy: 0.44698822498321533  Loss:  256.6766662597656
------------Solution  5 -------------
Accuracy: 0.5457509160041809  Loss:  280.40386962890625
------------Solution  6 -------------
Accuracy: 0.4546830654144287  Loss:  252.7967071533203
------------Solution